# Лабораторная работа №3
## Выполнил студент группы ФИО ГРУППА

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [11]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [3]:
import time

def prefix(s):
    v = [0]*len(s)
    for i in range(1,len(s)):
        k = v[i-1]
        while k > 0 and s[k] != s[i]:
            k = v[k-1]
        if s[k] == s[i]:
            k = k + 1
        v[i] = k
    return v


def kmp(s,t):
    index = -1
    f = prefix(s)
    k = 0
    for i in range(len(t)):
        while k > 0 and s[k] != t[i]:
            k = f[k-1]
        if s[k] == t[i]:
            k = k + 1
        if k == len(s):
            index = i - len(s) + 1
            break

    return index

haystack = input("Строка: ")
needle = input("Подстрока: ")
time_start = time.perf_counter()
print(f"index = {kmp(needle, haystack)}")
time_end = time.perf_counter()
print(f"Время работы алгоритма = {time_end-time_start}")

Строка: abcab
Подстрока: ca
index = 2
Время работы алгоритма = 0.0002062999999452586


#### Упрощенный алгоритм Бойера-Мура

In [4]:
a = input("Строка: ")
t = input("Подстрока: ")

def abm(a, t):
        # Этап 1: формирование таблицы смещений
        S = set()  # уникальные символы в образе
        M = len(t) # число символов в образе
        d = {}     # словарь смещений
        for i in range(M-2, -1, -1): # итерации с предпоследнего символа
            if t[i] not in S:        # если символ еще не добавлен в таблицу
                d[t[i]] = M-i-1
                S.add(t[i])
        if t[M-1] not in S:     # отдельно формируем последний символ
            d[t[M-1]] = M
        d['*'] = M              # смещения для прочих символов
        # Этап 2: поиск образа в строке
        N = len(a)
        if N >= M:
            i = M-1       # счетчик проверяемого символа в строке
            while(i < N):
                k = 0
                j = 0
                flBreak = False
                for j in range(M-1, -1, -1):
                    if a[i-k] != t[j]:
                        if j == M-1:
                            off = d[a[i]] if d.get(a[i], False) else d['*']  # смещение, если не равен последний символ образа
                        else:
                            off = d[t[j]]   # смещение, если не равен не последний символ образа

                        i += off    # смещение счетчика строки
                        flBreak = True  # если несовпадение символа, то flBreak = True
                        break
                    k += 1          # смещение для сравниваемого символа в строке
                if not flBreak:          # если дошли до начала образа, значит, все его символы совпали
                    return f"образ найден по индексу {i-k+1}"
            else:
                return "образ не найден"
        else:
            return "образ не найден"
        
time_start = time.perf_counter()
print(abm(a, t))
time_end = time.perf_counter()
print(f"Время работы алгоритма = {time_end-time_start}")

Строка: abcab
Подстрока: abc
образ найден по индексу 0
Время работы алгоритма = 0.0002926999999885993


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [11]:
from heapq import heappush, heappop
from datetime import datetime as t
import random
import math
import copy
def solve(data):
    sum = 0
    zero_coord = 0
    #проверка на наличие решения
    for i in range(len(data)):
        if data[i] == 0:
            zero_coord = i
            continue
        for j in range(i + 1, len(data)):
            if data[i] > data[j] and data[j] != 0:
                sum += 1
    if len(data) % 3 != 0:
        sum += zero_coord // math.sqrt(len(data)) + 1
    if sum % 2 != 0:
        return []
    #есть наличие, решаем
    else:
        start = chain(data)
        temp = copy.deepcopy(data)
        temp.sort()
        del temp[0]
        temp.append(0)
        end = chain(temp)
        result = a_star(start, end.last_node())
        for node in result.history:
            print(node)
        print(result)
        return 1
    
def a_star(start_chain, goal_node):
    node_hash = {}
    chains_queue = []
    heappush(chains_queue, start_chain)
    while chains_queue:
        cur_chain = heappop(chains_queue)
        cur_node = cur_chain.last_node()
        if cur_node == goal_node:
            return cur_chain
        node_hash[cur_node] = cur_chain.g()
        for chain in cur_chain.get_neighbours():
            if chain.last_node() in node_hash:
                if chain.g() >= node_hash[chain.last_node()]:
                    continue
                node_hash[chain.last_node()] = chain.g()
            heappush(chains_queue, chain)

def manh_dst_matrix(a, b, n):
    return abs(a % n - b % n) + abs(a // n - b // n)

class chain:
    # переопределяем str чтобы вывелась норм матрица
    def __str__(self):
        i = 0
        sstr = ""
        while i < self.size * self.size:
            sstr += str(self.board_state[i]) + " "
            if i % self.size == self.size - 1:
                sstr += "\n"
            i += 1
        return sstr
    
    def __init__(self, board_state, history=[]):
        self.board_state = list(board_state)
        self.size = int(math.sqrt(len(board_state)))
        self.history = history
        self.quad_size = int(self.size * self.size)
    # считаем манхэтеннское расстояние
    def manh_dst(self):
        dst = 0
        for i in range(0, self.quad_size):
            dst += manh_dst_matrix((self.board_state[i] - 1) % self.quad_size, i, self.size)
        return int(dst)
    #проверяем можно ли за одни ход закончить или же прибавляем 2
    def last_move(self):
        if self.board_state[-1] == self.quad_size - 1 or self.board_state[-1] == self.quad_size - self.size:
            return 0
        return 2
    # эврестическое предположение о расстоянии от текущей вершины до конечной
    def h(self):
        return self.manh_dst() + self.last_move()
    # сумма расстояний от начальной вершины до текущей
    def g(self):
        return len(self.history)
    # вычисление эвристической функции
    def f(self):
        return self.h() # + self.g() , если хотим найти оптимальный маршрут, но потребуется гораздо больше времени
    
    def last_node(self):
        return str(self.board_state)
    # переопределение __It__ для сравнения двух цепей (сравниваются по значению эвристической функции)
    def __lt__(self, other):
        return self.f() < other.f()
    
    def get_neighbours(self):
        neighs = []
        zero_coord = self.board_state.index(0)

        # проверяем соседей и создаем новые ветви 
        if zero_coord + 1 < self.size * self.size and manh_dst_matrix(zero_coord, zero_coord + 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + 1] = new_state[zero_coord + 1], new_state[zero_coord]
            neighs.append(chain(new_state, self.history + [self]))

        if zero_coord - 1 >= 0 and manh_dst_matrix(zero_coord, zero_coord - 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - 1] = new_state[zero_coord - 1], new_state[zero_coord]
            neighs.append(chain(new_state, self.history + [self]))

        if zero_coord + self.size < self.size * self.size and manh_dst_matrix(zero_coord, zero_coord + self.size,
                                                                       self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + self.size] = new_state[zero_coord + self.size], new_state[
                zero_coord]
            neighs.append(chain(new_state, self.history + [self]))

        if zero_coord - self.size >= 0 and manh_dst_matrix(zero_coord, zero_coord - self.size, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - self.size] = new_state[zero_coord - self.size], new_state[
                zero_coord]
            neighs.append(chain(new_state, self.history + [self]))

        return neighs
t1 = t.now()
kek = [i for i in range(16)]
random.shuffle(kek)
#print(solve([8, 13, 4, 15, 7, 1, 10, 2, 0, 11, 12, 3, 5, 6, 14, 9]))
print(kek)
print(solve(kek))
t2 = t.now()
delta = t2 - t1
print(delta)

[8, 13, 4, 15, 7, 1, 10, 2, 0, 11, 12, 3, 5, 6, 14, 9]
8 13 4 15 
7 1 10 2 
0 11 12 3 
5 6 14 9 

8 13 4 15 
7 1 10 2 
5 11 12 3 
0 6 14 9 

8 13 4 15 
7 1 10 2 
5 11 12 3 
6 0 14 9 

8 13 4 15 
7 1 10 2 
5 11 12 3 
6 14 0 9 

8 13 4 15 
7 1 10 2 
5 11 12 3 
6 14 9 0 

8 13 4 15 
7 1 10 2 
5 11 12 0 
6 14 9 3 

8 13 4 15 
7 1 10 2 
5 11 0 12 
6 14 9 3 

8 13 4 15 
7 1 10 2 
5 11 9 12 
6 14 0 3 

8 13 4 15 
7 1 10 2 
5 11 9 12 
6 14 3 0 

8 13 4 15 
7 1 10 2 
5 11 9 0 
6 14 3 12 

8 13 4 15 
7 1 10 2 
5 11 0 9 
6 14 3 12 

8 13 4 15 
7 1 0 2 
5 11 10 9 
6 14 3 12 

8 13 4 15 
7 1 2 0 
5 11 10 9 
6 14 3 12 

8 13 4 0 
7 1 2 15 
5 11 10 9 
6 14 3 12 

8 13 0 4 
7 1 2 15 
5 11 10 9 
6 14 3 12 

8 13 2 4 
7 1 0 15 
5 11 10 9 
6 14 3 12 

8 13 2 4 
7 1 10 15 
5 11 0 9 
6 14 3 12 

8 13 2 4 
7 1 10 15 
5 11 9 0 
6 14 3 12 

8 13 2 4 
7 1 10 0 
5 11 9 15 
6 14 3 12 

8 13 2 4 
7 1 0 10 
5 11 9 15 
6 14 3 12 

8 13 2 4 
7 1 9 10 
5 11 0 15 
6 14 3 12 

8 13 2 4 
7 1 9 10 
5 11 15 0 
6 14 3 12 


### Вывод